In [3]:
# A1_capture.py
import cv2, os, time

save_dir = "calib_phonecam"
os.makedirs(save_dir, exist_ok=True)

cap = cv2.VideoCapture(1)  # change to 1 if you have multiple cams
if not cap.isOpened():
    raise RuntimeError("Webcam not found")

print("Press SPACE to save a frame, ESC to exit.")
count = 0
while True:
    ok, frame = cap.read()
    if not ok: break

    txt = f"Saved: {count}  |  SPACE=save  ESC=quit"
    cv2.putText(frame, txt, (12,30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255,255,255), 2)
    cv2.imshow("Capture calibration views", frame)

    key = cv2.waitKey(1) & 0xFF
    if key == 27:      # ESC
        break
    elif key == 32:    # SPACE
        fname = os.path.join(save_dir, f"img_{count:02d}.jpg")
        cv2.imwrite(fname, frame)
        print("Saved", fname)
        count += 1
        time.sleep(0.15)  # avoid double taps

cap.release()
cv2.destroyAllWindows()


Press SPACE to save a frame, ESC to exit.
Saved calib_phonecam\img_00.jpg
Saved calib_phonecam\img_01.jpg
Saved calib_phonecam\img_02.jpg
Saved calib_phonecam\img_03.jpg
Saved calib_phonecam\img_04.jpg
Saved calib_phonecam\img_05.jpg
Saved calib_phonecam\img_06.jpg
Saved calib_phonecam\img_07.jpg
Saved calib_phonecam\img_08.jpg
Saved calib_phonecam\img_09.jpg
Saved calib_phonecam\img_10.jpg
Saved calib_phonecam\img_11.jpg
Saved calib_phonecam\img_12.jpg
Saved calib_phonecam\img_13.jpg
Saved calib_phonecam\img_14.jpg
Saved calib_phonecam\img_15.jpg
Saved calib_phonecam\img_16.jpg
Saved calib_phonecam\img_17.jpg
Saved calib_phonecam\img_18.jpg
Saved calib_phonecam\img_19.jpg


In [5]:
import cv2, glob, numpy as np

pattern_size = (8,6)  # inner corners if 9x7 squares printed
square_size = 1.0     # any unit (e.g., 1.0 or cm)

objp = np.zeros((pattern_size[0]*pattern_size[1],3), np.float32)
objp[:,:2] = np.mgrid[0:pattern_size[0],0:pattern_size[1]].T.reshape(-1,2) * square_size

objpoints, imgpoints = [], []
images = glob.glob("calib_phonecam/*.jpg")

for fname in images:
    img = cv2.imread(fname)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    ret, corners = cv2.findChessboardCorners(gray, pattern_size, None)
    if ret:
        corners = cv2.cornerSubPix(gray, corners, (11,11), (-1,-1),
                                   (cv2.TERM_CRITERIA_EPS+cv2.TERM_CRITERIA_MAX_ITER,30,0.001))
        objpoints.append(objp)
        imgpoints.append(corners)
        cv2.drawChessboardCorners(img, pattern_size, corners, ret)
        cv2.imshow("Corners", img)
        cv2.waitKey(200)
cv2.destroyAllWindows()

# Calibration
ret, K, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)

print("\nCamera Matrix (K):\n", K)
print("Distortion Coefficients:\n", dist.ravel())

# Compute reprojection error
total_err = 0
for i in range(len(objpoints)):
    imgpoints2, _ = cv2.projectPoints(objpoints[i], rvecs[i], tvecs[i], K, dist)
    err = cv2.norm(imgpoints[i], imgpoints2, cv2.NORM_L2)/len(imgpoints2)
    total_err += err
mean_err = total_err/len(objpoints)
print("Mean Reprojection Error (px):", mean_err)

# Save results
np.savez("calib_results_phonecam.npz", K=K, dist=dist, shape=gray.shape[::-1])


Camera Matrix (K):
 [[669.32498578   0.         632.19017391]
 [  0.         663.45544598 374.99341361]
 [  0.           0.           1.        ]]
Distortion Coefficients:
 [ 0.1405699  -0.15882257  0.01588326 -0.00455049 -0.01301423]
Mean Reprojection Error (px): 0.22531428953944413
